In [5]:
from exif import Image
from PIL import Image
from PIL.ExifTags import TAGS
from csv import DictWriter
import pandas as pd
import os

def img_filepaths():
    directory = "./images"
    filepaths = []
    for image in os.listdir(directory):
        i = os.path.join(directory, image)
        if os.path.isfile(i):
            filepaths.append(i)
    return filepaths

def dict_convert():
# Function that takes in return of img_filepaths() and outputs dictionaries of metadata from each jpeg file. 
    dict_list = []
    header = []
    image_names = img_filepaths()
    for image in image_names:
        try:
            image_file = Image.open(image)
            exifdata = image_file.getexif()
            #maps pillow tags onto id fields and prints
            file_dict = {}
            for tag_id in exifdata:
                # get the tag name, instead of human unreadable tag id
                tag = TAGS.get(tag_id, tag_id)
                data = exifdata.get(tag_id)
                # decode bytes 
                if isinstance(data, bytes):
                    data = data.decode()
                #add tags into header list
                header.append(tag)
                file_dict[tag] = data
            dict_list.append(file_dict)
        except:
            pass
        
    return dict_list
dict_list = dict_convert()
field_names = dict_list[1].keys()


with open("./data/meta_data.csv", 'w',newline='') as csvfile:
    writer = DictWriter(csvfile, fieldnames=field_names, extrasaction='ignore')
    writer.writeheader()
    writer.writerows(dict_list)